# Final Project Phase 2 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 2 of the Final Project. Answer all statements addressed below as specified in the instructions for the project, covering all necessary details. Please be clear and concise in your answers. Each response should be at most 3 sentences. Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

# Data Collection and Cleaning
You are required to provide data collection and cleaning for the three (3) minimum datasets. Create a function for each of the following sections that reads or scrapes data from a file or website, manipulate and cleans the parsed data, and writes the cleaned data into a new file. 

Make sure your data cleaning and manipulation process is not too simple. Performing complex manipulation and using modules not taught in class shows effort, which will increase the chance of receiving full credit.


## Data Sources
Include sources (as links) to your datasets. Add any additional data sources if needed. Clearly indicate if a data source is different from one submitted in your Phase I, as we will check that it satisfies the requirements.
*   Downloaded Dataset Source: https://simfin.com/data/bulk
*   Web Collection #1 Source: CHANGED https://www.barchart.com/etfs-funds/performance/percent-change/advances?timeFrame=10y
*   Web Collection #2 Source: https://data.nasdaq.com/api/v3/datasets/



## Downloaded Dataset Requirement

Fill in the predefined functions with your data scraping/parsing code. You may modify/rename each function as you seem fit, but you must provide at least 3 separate functions that clean each of your required datasets.


In [ ]:
!pip install selenium 
!pip install webdriver-manager

In [ ]:
import pandas as pd
import numpy as np
import json
from pprint import pprint
def data_parser():
    df = pd.read_csv('us-income-quarterly 2.csv', delimiter = ';')
    df = df.drop(['Depreciation & Amortization','SimFinId','Currency', 'Report Date', 'Publish Date', 'Restated Date', 'Abnormal Gains (Losses)', 'Net Extraordinary Gains (Losses)'], axis=1)
    df = df.fillna(0)
    data = {}
    fiscal_periods = {'Q1': .1, 'Q2': .2, 'Q3': .3, 'Q4': .4}
    for i,j in df.groupby('Ticker', as_index=False):
        data[i] = j.set_index(j['Fiscal Year'].values + [fiscal_periods[i] for i in j['Fiscal Period'].values]).drop(['Fiscal Year', 'Fiscal Period'], axis=1)
        data[i] = data[i].to_json()
    data['Tickers'] = list(data.keys())
    json.dump(data, open('corporate.json', 'w'))
    pprint(data)
    return 'Complete'
############ Function Call ############
data_parser()

## Web Collection Requirement \#1


In [ ]:
# NOTE: THE URL HAS CHANGED FOR THIS SECTION
#The links I will now be scraping are the following: 
#https://www.barchart.com/etfs-funds/performance/percent-change/advances?timeFrame=10y
#https://www.barchart.com/etfs-funds/performance/percent-change/declines?timeFrame=10y&orderBy=percentChange&orderDir=asc
#Each generate seperate DataFrames which I will use for analysis
#Note- the method of scraping both these websites are the same, only the link changes. Therefore I am not creating two different methods.
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from pprint import pprint
from selenium import webdriver
import re
def web_parser1():
    url= "https://www.barchart.com/etfs-funds/performance/percent-change/declines?timeFrame=10y&orderBy=percentChange&orderDir=asc"
    title_name =re.findall('https://www.barchart.com/etfs-funds/performance/(.+)\?', url)[0].replace('/', ' ').title()
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    html = driver.page_source
    df = pd.read_html(html)
    df = df[-1].set_index('Symbol').drop(['Time','Links'], axis=1)
    df.to_csv(f'{title_name}.csv')
    pprint(df)
    return "Complete"
    

############ Function Call ############
web_parser1()

## Web Collection Requirement \#2

In [ ]:
import requests as rq
import pandas as pd
import numpy as np
def web_parser2():
    cpi = rq.get("https://data.nasdaq.com/api/v3/datasets/RATEINF/CPI_USA/data.json?api_key=gnYkCy78FiQnCZKx7h9z").json()
    df = pd.DataFrame(cpi['dataset_data']['data']).set_index(0).iloc[::-1]
    df.index = pd.to_datetime(df.index)
    df['Log % Change'] = np.log(df[1]).pct_change().multiply(100)
    df['Log Change'] = np.log(df[1]).diff(1)
    df['Quarterly Rolling Log Change'] = df['Log Change'].rolling(3).mean()
    df['Yearly Rolling Log Change'] = df['Log Change'].rolling(6).mean()
    df = df[1044:]
    df.to_csv('cpi.csv')
    pprint(df)
    return "Complete"

############ Function Call ############
web_parser2()

## Additional Dataset Parsing/Cleaning Functions

Write any supplemental (optional) functions here.

In [ ]:
from scipy.signal import lfilter
def nasdaq_scrape_ustreasury(dataset):
    df = pd.DataFrame(rq.get(f"https://data.nasdaq.com/api/v3/datasets/{dataset}/data.json?api_key=gnYkCy78FiQnCZKx7h9z").json()['dataset_data']['data'], columns=rq.get(f"https://data.nasdaq.com/api/v3/datasets/{dataset}/data.json?api_key=gnYkCy78FiQnCZKx7h9z").json()['dataset_data']['column_names']).set_index('Date').iloc[::-1]
    df.index = pd.to_datetime(df.index)
    return df

def nasdaq_scrape_yale(dataset):
    df = pd.DataFrame(rq.get(f"https://data.nasdaq.com/api/v3/datasets/{dataset}/data.json?api_key=gnYkCy78FiQnCZKx7h9z").json()['dataset_data']['data'], columns=rq.get(f"https://data.nasdaq.com/api/v3/datasets/{dataset}/data.json?api_key=gnYkCy78FiQnCZKx7h9z").json()['dataset_data']['column_names']).set_index('Year').iloc[::-1]
    df.index = pd.to_datetime(df.index)
    return df

def noise_filter(data):
    n = 30
    b = [1.0 / n] * n
    a = 1
    noiseless= pd.DataFrame(lfilter(b,a,data))
    noiseless.index = data.index
    return noiseless.dropna()

def extra_source1():
    real_yield = nasdaq_scrape_ustreasury('USTREASURY/REALYIELD')
    real_yield = real_yield.drop('30 YR', axis=1)
    real_yield = real_yield.loc[real_yield['20 YR'].dropna().index[0]:]
    pprint(real_yield)
    real_yield.to_csv('real_yield.csv')
    bill_rates = nasdaq_scrape_ustreasury('USTREASURY/BILLRATES')
    bill_rates = bill_rates.drop(['52 Wk Coupon Equiv', '8 Wk Bank Discount Rate', '8 Wk Coupon Equiv', '52 Wk Bank Discount Rate' ], axis=1)
    bill_rates.to_csv('bill_rates.csv')
    pprint(bill_rates)
    spy_composite = nasdaq_scrape_yale('YALE/SPCOMP').fillna(0)
    spy_composite.to_csv('spy_composite')
    pprint(spy_composite)
    spy_confidence_individual = noise_filter(np.log(nasdaq_scrape_ustreasury('YALE/US_CONF_INDEX_VAL_INDIV'))[6:].pct_change().rolling(3).mean())
    spy_confidence_individual.to_csv('spy_confidence_individual.csv')
    pprint(spy_confidence_individual)
    spy_confidence_institutional = noise_filter(np.log(nasdaq_scrape_ustreasury('YALE/US_CONF_INDEX_VAL_INST'))[22:].pct_change().rolling(3).mean())
    spy_confidence_institutional.to_csv('spy_confidence_institutional')
    pprint(spy_confidence_institutional)
    spy_crash_individual = noise_filter(np.log(nasdaq_scrape_ustreasury('YALE/US_CONF_INDEX_CRASH_INDIV'))[3:].pct_change().rolling(3).mean())
    spy_crash_individual.to_csv('spy_crash_individual.csv')
    pprint(spy_crash_individual)
    spy_crash_institutional = noise_filter(np.log(nasdaq_scrape_ustreasury('YALE/US_CONF_INDEX_CRASH_INST'))[21:].pct_change().rolling(3).mean())
    spy_crash_institutional.to_csv('spy_crash_institutional.csv')
    pprint(spy_crash_institutional)
    housing_pricing_index = noise_filter(np.log(nasdaq_scrape_ustreasury('YALE/NHPI'))[627:].diff())
    housing_pricing_index.to_csv('housing_pricing_index.csv')
    pprint(housing_pricing_index)
    spy_dividend = noise_filter(np.log(nasdaq_scrape_ustreasury('MULTPL/SP500_DIV_YIELD_MONTH')[1548:]).pct_change().fillna(0))
    spy_pe_ratio = noise_filter(nasdaq_scrape_ustreasury('MULTPL/SP500_PE_RATIO_MONTH')[1548:].pct_change().fillna(0))
    schiller_pe_ratio =noise_filter(np.log(nasdaq_scrape_ustreasury('MULTPL/SHILLER_PE_RATIO_MONTH')[1548:]).pct_change().fillna(0))
    spy_monthly_inflation = noise_filter(np.log(nasdaq_scrape_ustreasury('MULTPL/SP500_INFLADJ_MONTH')[1548:]).pct_change().fillna(0))
    multpl = pd.concat([spy_dividend, spy_pe_ratio, schiller_pe_ratio, spy_monthly_inflation], axis=1, keys = ['spy_dividend', 'spy_pe_ratio', 'schiller_pe_ratio', 'spy_monthly_inflation']).fillna(0)
    multpl.to_csv('multpl.csv')
    pprint(multpl)
    return 'Complete'

    
############ Function Call ############
extra_source1()


#Inconsistencies
For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. NaN and missing data- nearly every dataset used in this projection contains missing data. There are two ways to handle this- dropping data or filling data. Determining which option is better can only be determined on a case byc ase basis, however the general note used throughout this project was to drop values for data that is non-stationary (will explain later) for analysis and use fillna(0) for stationary datasets.  

2. Uneven Indices- Despite coming from the same dataset or api, most of these datasets have different indexes. There were two steps to this problem. First, the "Date" or "Year" values needed to be converted into DateTime objects so that they can be set as indices. Second, when creating one dataframe the indices needed to start and end at the same index. Therefore, this project required manually going through each of the datasets and locating a range of dates where each dataset had proper values. This is why some datasets have [n:] at the end. Since pandas does not handle ragged DataFrames, this was necessary during preprocessing. 

3. Unequal Index Intervals- Some datasets (predominantly those from USTreasury) contained indices with uneven intervals. I.E there were only 4 data points from 1960-2000 but 700+ datapoints from 2000-present in the Time Series Data. These unconsistant intervals posed a problem in that pattenrs and trends can not be extrapolated as the data is not formatted correctly. The main solution is to identify these instances and truncate the time series data. 

4. Stationarity- Most (if not all) of the data is pretty useless preprocessed. That is because none of the time series data is stationary. "A stationary process has the property that the mean, variance and autocorrelation structure do not change over time." (Engineering Statistics Handbook). For time series analysis stationarity is crucial as forecasting and trends can not be identified if data is not stationary. While there are any methods to make data stationary, three methods where implemented in this project. First, converting data to logarithmic data using np.log can make data stationary. Second, converting the time series model to a difference-model such that each value is the change in the discrete intervals bounds the time series model to a certain mean (generally zero). This can be done using .diff() with pandas DataFrames. Lastly, .pct_change() converts DataFrames into the percent change between discrete intervals of the index. The latter method often is the best method as % changes generally reflect seasonal or event-driven patterns. 

5. Noise- Another property that makes most of this data meaningless at first is noise. Noisy data involves the statistical variation or "noise" in data which often renders the general trend moot. Noise within these datasets are reflected by the varying change in daily data which as a results makes the stationary data extremely variable and "noisy". There were two methods implemented to reduce noise within the data sets. First, using rolling means (weekly or monthly) smooths the time series data and reduces daily variability. This can be achieved by .rolling(n).mean() for DataFrames. Lastly, scipy.filter provides various signal processing methods to filter noisy data. I used lfilter() to eliminate noise. While I do not fully understand the backend of this method, lfilter can be applied to 1-D data only and is forward filtering based off a filtering function of your choise 'B'. Not knowing much, I used a previous example for this method.  See scipy.signal.lfilter for more details.
